In [1]:
import tensorflow as tf
import tensorflow.keras.backend as kb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, MaxPooling1D, Conv1D, LSTM
from tensorflow.keras.backend import ctc_batch_cost
import numpy as np

device_hasgpu = tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)
if device_hasgpu:
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
    sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [5]:
def ctc_custom(prediction_batch, label_batch):
    input_length = kb.map_fn(lambda x: np.array([20]), prediction_batch)
    label_length = kb.map_fn(lambda x: np.array([x.shape[1]]), label_batch)
    return ctc_batch_cost(label_batch, prediction_batch, input_length, label_length)

In [6]:
model = Sequential()

model.add(Conv1D(32, 3,
          padding="valid",
          activation="relu", 
          input_shape=(200,1))) # here input shape is set
model.add(MaxPooling1D(pool_size=5)) # just to make it smaller for now

model.add(LSTM(32,return_sequences=True))

model.add(Dense(5, activation="softmax"))

model.compile(optimizer="adam", loss=ctc_custom, metrics=['accuracy'])


TypeError: Expected binary or unicode string, got Dimension(5)